<a href="https://colab.research.google.com/github/noktavirgul/Dil-analizi-ile-alzheimer-risk-tespiti/blob/main/modal_egitim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install sentence-transformers xgboost imbalanced-learn joblib openpyxl


In [ ]:
import os, json, joblib
import numpy as np
import pandas as pd
from collections import Counter

from sentence_transformers import SentenceTransformer
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, f1_score, accuracy_score
from xgboost import XGBClassifier

# ===== Ayarlar =====
EXCEL_FILE = "alzheimer.xlsx"   # <-- dosya adını buraya yazın
TEXT_COL   = "Metin"

label_cols = [
    "cumle_uzunlugu",
    "kelime_sikligi",
    "zamir_hatalari",
    "baglac_hatalari",
    "anlam_belirsizligi",
    "sozcuk_seciminde_tutarsizlik"
]

# SBERT modeli
EMB_MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"

# Risk kuralı
MIN_POS_NEEDED = 2
RANDOM_STATE = 42


In [ ]:
df = pd.read_excel(EXCEL_FILE)
df = df[[TEXT_COL] + label_cols].dropna()

print("Örnek veri:")
display(df.head())

# Etiketlerin 0/1 olduğundan emin olalım
for c in label_cols:
    bad = set(df[c].unique()) - {0,1}
    if bad:
        raise ValueError(f"{c} sütununda 0/1 dışı değerler var: {bad}")

# Her etiketin dağılımı
for c in label_cols:
    print(f"{c}: 1 sayısı = {df[c].sum()}, 0 sayısı = {(df[c]==0).sum()}")

Örnek veri:


,Metin,cumle_uzunlugu,kelime_sikligi,zamir_hatalari,baglac_hatalari,anlam_belirsizligi,sozcuk_seciminde_tutarsizlik
0,Ben… ben neredeyim?,1,1,0,0,0,0
1,Dün doktora gideceğim.,1,0,0,0,1,0
2,Oğlumun adını unuttum.,1,0,0,0,0,0
3,Kahvaltıda ayakkabımı yedim.,1,0,0,0,1,1
4,Kapıyı kilitledim mi? Kilitledim mi?,1,0,0,0,0,0


cumle_uzunlugu: 1 sayısı = 1368, 0 sayısı = 4082
kelime_sikligi: 1 sayısı = 734, 0 sayısı = 4716
zamir_hatalari: 1 sayısı = 663, 0 sayısı = 4787
baglac_hatalari: 1 sayısı = 341, 0 sayısı = 5109
anlam_belirsizligi: 1 sayısı = 1525, 0 sayısı = 3925
sozcuk_seciminde_tutarsizlik: 1 sayısı = 1516, 0 sayısı = 3934


In [ ]:
df["risk_true"] = (df[label_cols].sum(axis=1) >= MIN_POS_NEEDED).astype(int)

X_text = df[TEXT_COL].astype(str).values
Y = df[label_cols].astype(int)
risk = df["risk_true"].values

# 70/15/15 bölme
sss1 = StratifiedShuffleSplit(n_splits=1, test_size=0.30, random_state=RANDOM_STATE)
train_idx, temp_idx = next(sss1.split(X_text, risk))

X_train_text, y_train = X_text[train_idx], Y.iloc[train_idx].reset_index(drop=True)
risk_train = risk[train_idx]

X_temp_text, y_temp = X_text[temp_idx], Y.iloc[temp_idx].reset_index(drop=True)
risk_temp = risk[temp_idx]

sss2 = StratifiedShuffleSplit(n_splits=1, test_size=0.50, random_state=RANDOM_STATE)
valid_idx, test_idx = next(sss2.split(X_temp_text, risk_temp))

X_valid_text, y_valid = X_temp_text[valid_idx], y_temp.iloc[valid_idx].reset_index(drop=True)
X_test_text,  y_test  = X_temp_text[test_idx],  y_temp.iloc[test_idx].reset_index(drop=True)

print("Boyutlar ->",
      "train:", len(X_train_text),
      "valid:", len(X_valid_text),
      "test:",  len(X_test_text))


Boyutlar -> train: 3815 valid: 817 test: 818


In [ ]:
model = SentenceTransformer(EMB_MODEL_NAME)

def embed(text_list, batch_size=256):
    return model.encode(list(text_list), batch_size=batch_size, show_progress_bar=True, convert_to_numpy=True)

X_train = embed(X_train_text)
X_valid = embed(X_valid_text)
X_test  = embed(X_test_text)

print("Embedding boyutları:", X_train.shape)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Embedding boyutları: (3815, 384)


In [ ]:
models = {}
pos_weights = {}

for col in label_cols:
    pos = int(y_train[col].sum())
    neg = int(len(y_train) - pos)
    if pos == 0:
        print(f"⚠ {col} etiketinde hiç 1 yok, model atlandı.")
        continue
    spw = neg / pos
    pos_weights[col] = spw

    clf = XGBClassifier(
        n_estimators=400,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=RANDOM_STATE,
        eval_metric="logloss",
        scale_pos_weight=spw,
        tree_method="hist"
    )
    clf.fit(X_train, y_train[col])
    models[col] = clf

print("scale_pos_weight:", {k: round(v,2) for k,v in pos_weights.items()})


scale_pos_weight: {'cumle_uzunlugu': 2.98, 'kelime_sikligi': 6.41, 'zamir_hatalari': 7.46, 'baglac_hatalari': 14.9, 'anlam_belirsizligi': 2.6, 'sozcuk_seciminde_tutarsizlik': 2.59}


In [ ]:
proba_valid = pd.DataFrame({c: models[c].predict_proba(X_valid)[:,1] for c in models})
proba_test  = pd.DataFrame({c: models[c].predict_proba(X_test)[:,1]  for c in models})

# Eksik etiketler varsa sıfırla
for c in label_cols:
    if c not in proba_valid.columns:
        proba_valid[c] = 0.0
    if c not in proba_test.columns:
        proba_test[c] = 0.0

proba_valid = proba_valid[label_cols]
proba_test  = proba_test[label_cols]


In [ ]:
def optimize_label_thresholds(y_true_df, proba_df, label_cols):
    thresholds = {}
    for c in label_cols:
        y_true = y_true_df[c].values
        p = proba_df[c].values
        best_t, best_score = 0.5, -1
        for t in np.linspace(0.1, 0.9, 17):
            yhat = (p >= t).astype(int)
            s = f1_score(y_true, yhat, zero_division=0)
            if s > best_score:
                best_score, best_t = s, t
        thresholds[c] = float(best_t)
    return thresholds

label_thresholds = optimize_label_thresholds(y_valid, proba_valid, label_cols)
label_thresholds


{'cumle_uzunlugu': 0.5,
 'kelime_sikligi': 0.30000000000000004,
 'zamir_hatalari': 0.65,
 'baglac_hatalari': 0.30000000000000004,
 'anlam_belirsizligi': 0.45000000000000007,
 'sozcuk_seciminde_tutarsizlik': 0.55}

In [ ]:
from itertools import product

def optimize_weights_and_threshold(y_valid_df, proba_valid_df, label_cols, label_thresholds, weight_candidates, min_pos_needed=2):
    true_risk = (y_valid_df[label_cols].sum(axis=1) >= min_pos_needed).astype(int)
    yhat_bin = pd.DataFrame({
        c: (proba_valid_df[c] >= label_thresholds.get(c, 0.5)).astype(int) for c in label_cols
    })

    best = {"score": -1}
    for weights in product(weight_candidates, repeat=len(label_cols)):
        w = pd.Series(weights, index=label_cols)
        weighted_score = (yhat_bin * w).sum(axis=1)
        for thr in np.unique(weighted_score):
            risk_pred = (weighted_score >= thr).astype(int)
            score = f1_score(true_risk, risk_pred, zero_division=0)
            if score > best["score"]:
                best = {"score": score, "weights": w.to_dict(), "risk_threshold": float(thr)}
    return best

best = optimize_weights_and_threshold(
    y_valid_df=y_valid,
    proba_valid_df=proba_valid,
    label_cols=label_cols,
    label_thresholds=label_thresholds,
    weight_candidates=(1.0, 1.5, 2.0),
    min_pos_needed=MIN_POS_NEEDED
)

print(">> En iyi F1:", round(best["score"], 4))
print(">> Seçilen ağırlıklar:", best["weights"])
print(">> Risk eşiği:", best["risk_threshold"])


>> En iyi F1: 0.7871
>> Seçilen ağırlıklar: {'cumle_uzunlugu': 1.0, 'kelime_sikligi': 1.0, 'zamir_hatalari': 1.5, 'baglac_hatalari': 1.0, 'anlam_belirsizligi': 1.0, 'sozcuk_seciminde_tutarsizlik': 1.0}
>> Risk eşiği: 1.5


In [ ]:
yhat_test_bin = pd.DataFrame({
    c: (proba_test[c] >= label_thresholds.get(c, 0.5)).astype(int) for c in label_cols
})

w = pd.Series(best["weights"])
risk_thr = best["risk_threshold"]

weighted_score_test = (yhat_test_bin * w).sum(axis=1)
risk_pred_test = (weighted_score_test >= risk_thr).astype(int)

true_risk_test = (y_test[label_cols].sum(axis=1) >= MIN_POS_NEEDED).astype(int)
print(classification_report(true_risk_test, risk_pred_test, digits=3))


              precision    recall  f1-score   support

           0      0.795     0.872     0.832       468
           1      0.803     0.700     0.748       350

    accuracy                          0.798       818
   macro avg      0.799     0.786     0.790       818
weighted avg      0.799     0.798     0.796       818



In [ ]:
SAVE_DIR = "alz_sbert_weighted"
os.makedirs(SAVE_DIR, exist_ok=True)

for c, clf in models.items():
    joblib.dump(clf, os.path.join(SAVE_DIR, f"xgb_{c}.joblib"))

export_cfg = {
    "emb_model_name": EMB_MODEL_NAME,
    "label_cols": label_cols,
    "label_thresholds": label_thresholds,
    "weights": best["weights"],
    "risk_threshold": best["risk_threshold"],
    "min_pos_needed": MIN_POS_NEEDED
}
with open(os.path.join(SAVE_DIR, "config.json"), "w") as f:
    json.dump(export_cfg, f, indent=2)

print("Kaydedildi:", SAVE_DIR)


Kaydedildi: alz_sbert_weighted


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np

# Gerekli dil analizi araçlarını yeniden başlatma
from zemberek import TurkishMorphology
import stanza
from sklearn.preprocessing import StandardScaler

# Modelinize uygun olarak kullanılan BERTurk tokenizer ve modeli
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
bert_model = AutoModel.from_pretrained("dbmdz/bert-base-turkish-cased")

# Zemberek ve Stanza
morphology_zemberek = TurkishMorphology.create_with_defaults()
try:
    stanza.download('tr', processors='tokenize,pos')
    nlp_stanza = stanza.Pipeline(lang='tr', processors='tokenize,pos', verbose=False) # verbose=False to reduce output
except Exception as e:
    print(f"Stanza modelini indirirken veya başlatırken bir hata oluştu: {e}")
    nlp_stanza = None


# Daha önce tanımladığınız özellik çıkarma fonksiyonları
def get_bert_embeddings_single(text):
    encoded_input = tokenizer(
        [text],
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )
    with torch.no_grad():
        model_output = bert_model(**encoded_input)
    return model_output.last_hidden_state[:, 0, :].numpy()

def extract_hybrid_features_single(text, morphology, nlp_stanza):
    features = []
    text_str = str(text) if pd.notna(text) else ""
    sentence_length = len(text_str)

    word_count_stanza = 0
    pronoun_count_stanza = 0
    conjunction_count_stanza = 0

    if nlp_stanza:
        doc = nlp_stanza(text_str)
        words_stanza = [word.text for sentence in doc.sentences for word in sentence.words]
        word_count_stanza = len(words_stanza)
        pronoun_count_stanza = sum(1 for sentence in doc.sentences for word in sentence.words if word.upos == 'PRON')
        conjunction_count_stanza = sum(1 for sentence in doc.sentences for word in sentence.words if word.upos == 'CCONJ')

    pronoun_count_zemberek = 0
    conjunction_count_zemberek = 0
    if word_count_stanza > 0:
        for word_text in words_stanza:
            try:
                analysis = morphology.analyze(word_text)
                if not analysis.is_empty():
                    if any('Pron' in str(ana) for ana in analysis):
                        pronoun_count_zemberek += 1
                    if any('Conj' in str(ana) for ana in analysis):
                        conjunction_count_zemberek += 1
            except Exception:
                pass

    features.append([
        sentence_length,
        word_count_stanza,
        pronoun_count_stanza,
        conjunction_count_stanza,
        pronoun_count_zemberek,
        conjunction_count_zemberek
    ])

    return np.array(features)

def predict_risk(sentence):
    if loaded_model is None or loaded_scaler is None or nlp_stanza is None:
        print("Model, ölçekleyici veya dil analizi araçları yüklenemedi. Lütfen kontrol edin.")
        return

    print(f"\nGirilen Cümle: '{sentence}'")

    # Adım 1: Dilbilgisel özellikleri çıkar
    hybrid_features = extract_hybrid_features_single(sentence, morphology_zemberek, nlp_stanza)

    # Adım 2: Özellikleri ölçekle
    scaled_features = loaded_scaler.transform(hybrid_features)

    # Adım 3: BERT embedding'lerini çıkar
    bert_embeddings = get_bert_embeddings_single(sentence)

    # Adım 4: Tüm özellikleri birleştir
    combined_features = np.hstack((bert_embeddings, scaled_features))

    # Adım 5: Her bir etiket için tahmin olasılıklarını al
    proba = {}
    for col, model in loaded_model.items():
         proba[col] = model.predict_proba(combined_features)[:, 1][0] # Olasılıkları al

    # Adım 6: Optimal eşikleri kullanarak tahmin yap
    y_pred_single = {}
    risk_indicator_count = 0
    for col in proba.keys():
        # Thresholds dictionary from cell MQJNR-LRIhC4 is needed here
        # Ensure 'thresholds' is accessible in this scope or pass it as an argument
        predicted_label = 1 if proba[col] >= thresholds[col] else 0
        y_pred_single[col] = predicted_label
        if predicted_label == 1:
            risk_indicator_count += 1

    # Adım 7: Risk kuralını uygula
    overall_risk = "Riskli" if risk_indicator_count >= 2 else "Risksiz"

    # Adım 8: Tahminleri okunabilir hale getir
    result = {}
    for label, prediction in y_pred_single.items():
        result[label] = "Riskli" if prediction == 1 else "Risksiz"

    result['Genel Risk Durumu'] = overall_risk

    return result

# Kullanıcıdan girdi alma döngüsü
while True:
    user_input = input("\nTest etmek için bir cümle girin (çıkmak için 'q' yazın): ")
    if user_input.lower() == 'q':
        print("Programdan çıkılıyor.")
        break

    prediction_result = predict_risk(user_input)
    if prediction_result:
        print("\n--- Analiz Sonucu ---")
        for k, v in prediction_result.items():
            print(f"- {k.replace('_', ' ').capitalize()}: {v}")

INFO:zemberek.morphology.turkish_morphology:TurkishMorphology instance initialized in 12.312947034835815


2025-08-15 01:09:14,216 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 12.312947034835815



INFO:stanza:Downloaded file to /root/stanza_resources/resources.json


2025-08-15 01:09:14,303 - stanza - INFO
Msg: Downloaded file to /root/stanza_resources/resources.json



2025-08-15 01:09:14,316 - stanza - WARNING
Msg: Language tr package default expects mwt, which has been added



...

2025-08-15 01:09:14,879 - stanza - INFO
Msg: File exists: /root/stanza_resources/tr/backward_charlm/conll17.pt



INFO:stanza:File exists: /root/stanza_resources/tr/forward_charlm/conll17.pt


2025-08-15 01:09:14,932 - stanza - INFO
Msg: File exists: /root/stanza_resources/tr/forward_charlm/conll17.pt



INFO:stanza:Finished downloading models and saved to /root/stanza_resources


2025-08-15 01:09:14,934 - stanza - INFO
Msg: Finished downloading models and saved to /root/stanza_resources


Girilen Cümle: 'Kimseye kötülük yapmadılar biz de yapmadık.'

--- Analiz Sonucu ---
- Cumle uzunlugu: Risksiz
- Kelime sikligi: Risksiz
- Zamir hatalari: Risksiz
- Baglac hatalari: Risksiz
- Anlam belirsizligi: Risksiz
- Sozcuk seciminde tutarsizlik: Risksiz
- Genel risk durumu: Risksiz


In [ ]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving alzheimer.xlsx to alzheimer.xlsx
User uploaded file "alzheimer.xlsx" with length 241099 bytes


In [ ]:
# Veriyi eğitim ve test setlerine ayırma
X = df[TEXT_COL]
y = df[label_cols]

# Stratified sampling kullanarak etiket dağılımını koruma
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)

# İlk split'i alma
train_index, test_index = next(sss.split(X, y))

X_train, X_test = X.iloc[train_index], X.iloc[test_index]
y_train, y_test = y.iloc[train_index], y.iloc[test_index]

print(f"Eğitim seti boyutu: {X_train.shape[0]}")
print(f"Test seti boyutu: {X_test.shape[0]}")

# Eğitim ve test setlerinde etiket dağılımını kontrol etme (isteğe bağlı)
print("\nEğitim seti etiket dağılımı:")
display(y_train.sum().to_frame(name='Train Count'))

print("\nTest seti etiket dağılımı:")
display(y_test.sum().to_frame(name='Test Count'))

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.